In [1]:
# Install necessary packages if not already installed
!pip install BeautifulSoup
!pip install pymysql
!pip install python-dotenv
!pip install pandas
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable
  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      Traceback (most recent call last):
        File "C:\Users\spenc\AppData\Roaming\Python\Python313\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "C:\Users\spenc\AppData\Roaming\Python\Python313\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\spenc\AppData\Roaming\Python\Python313\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\spenc\AppData\Local\Temp\pip-build-env-h403ttdk\overlay\Lib\site-packages\setuptools\build_meta.py", li

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [21]:
# Import necessary modules
import os
import requests
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from dotenv import load_dotenv

In [22]:
# Load environment variables from .env (data base credentials)
load_dotenv()

True

In [23]:
# URL for the Wikipedia page for the S&P 500 companies.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

In [24]:
# Fetch the webpage
response = requests.get(url)
response.raise_for_status()  # Raise an exception for HTTP errors

In [26]:
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find the table with id "constituents"
table = soup.find("table", {"id": "constituents"})
if table is None:
    raise Exception("Could not find the 'constituents' table on the page.")

In [27]:
# Attempt to extract headers using <thead> if available, otherwise use the first row of the table.
thead = table.find("thead")
if thead:
    headers = [th.get_text(strip=True) for th in thead.find_all("th")]
else:
    # Fallback: Use the first row (assumed to be header) from the table
    first_row = table.find("tr")
    if first_row:
        headers = [cell.get_text(strip=True) for cell in first_row.find_all(["th", "td"])]
    else:
        raise Exception("No header row found in the table.")

In [29]:
# Extract all data rows (skip header row if it's within <tbody>)
tbody = table.find("tbody")
if tbody:
    data_rows = tbody.find_all("tr")
else:
    # If there is no <tbody>, get all <tr> elements and skip the first row (headers)
    data_rows = table.find_all("tr")[1:]

    rows = []
for tr in data_rows:
    # Get all cell values from the row (using <td> elements)
    cells = tr.find_all("td")
    row = [cell.get_text(strip=True) for cell in cells]
    if row:
        rows.append(row)

In [31]:
# Create a DataFrame from the scraped data
df_sp500 = pd.DataFrame(rows, columns=headers)
print("Sample scraped data from Wikipedia:")
print(df_sp500.head())

Sample scraped data from Wikipedia:
  Symbol             Security              GICSSector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

          CIK      Founded  
0  0000066740         1902  
1  0000091142         1916  
2  0000001800         1888  
3  0001551152  2013 (1888)  
4  0001467373         1989  


In [33]:
# Establish a database connection using credentials from your .env file
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

if None in (DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME):
    raise Exception("One or more database credentials are not set in the .env file.")

# Create a SQLAlchemy engine for MySQL
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [34]:
# Write the DataFrame to a MySQL table named 'raw_wikipedia_sp500'
df_sp500.to_sql(name="raw_wikipedia_sp500", con=engine, if_exists="replace", index=False)
print("Scraped Wikipedia data loaded successfully into 'raw_wikipedia_sp500' table.")

Scraped Wikipedia data loaded successfully into 'raw_wikipedia_sp500' table.
